### **Note:** 
Before executing the cell, replace the path "*drive/MyDrive/Structure-From-Motion-Accurate-Optical-Flow-vs-Active-Vision*"
with your path to the project.

In [ ]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

%cd drive/MyDrive/Structure-From-Motion-Accurate-Optical-Flow-vs-Active-Vision

# MMflow installation
!pip3 install -U openmim
!mim install mmcv-full

!pip install mmflow
!pip install -e .


#ROS installation
!sudo sh -c 'echo "deb http://packages.ros.org/ros/ubuntu $(lsb_release -sc) main" > /etc/apt/sources.list.d/ros-latest.list'
!sudo apt-key adv --keyserver 'hkp://keyserver.ubuntu.com:80' --recv-key C1CF6E31E6BADE8868B172B4F42ED6FBAB17C654
!sudo apt update
!sudo apt install ros-noetic-desktop-full
!pip install --extra-index-url https://rospypi.github.io/simple/ rospy
!pip install --extra-index-url https://rospypi.github.io/simple/ tf2-ros
!pip install bagpy
!pip install cvbridge3


!pip install rospy_message_converter



/content/drive/MyDrive/Structure-From-Motion-Accurate-Optical-Flow-vs-Active-Vision
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu116/torch1.13.0/index.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/Structure-From-Motion-Accurate-Optical-Flow-vs-Active-Vision
  Preparing metadata (setup.py) ... done
ERROR: No .egg-info directory found in /tmp/pip-pip-egg-info-aakh7hnc
Executing: /tmp/apt-key-gpghome.X7nZDktqR7/gpg.1.sh --keyserver hkp://keyserver.ubuntu.com:80 --recv-key C1CF6E31E6BADE8868B172B4F42ED6FBAB17C654
gpg: key F42ED6FBAB17C654: "Open Robotics <info@osrfoundation.org>" not ch

In [ ]:
%cd drive/MyDrive/Structure-From-Motion-Accurate-Optical-Flow-vs-Active-Vision

!rm -rf __pycache__

import rospy
from scipy.spatial.transform import Rotation as R
import cv2
import torch
#import torchvision
torch.cuda.is_available = lambda: True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from mmflow.apis import init_model, inference_model
from mmflow.datasets import visualize_flow, write_flow

import time
import subprocess
import importlib
from rospy_message_converter import message_converter

from src.utils.utils_rosbag import *
from src.utils.utils_opticflow import *
from src.utils.utils_detection import *
from src.utils.utils_plot import *
from src.utils.utils_depth import *

#start roscore
!printf -- '#!/bin/bash\n' > ros_start.sh
!printf -- 'source /opt/ros/noetic/setup.bash\n' >> ros_start.sh
!printf -- '/opt/ros/noetic/bin/roscore' >> ros_start.sh
!chmod +x ros_start.sh
!cat ros_start.sh

prc = subprocess.Popen([os.getcwd()+"/ros_start.sh"])


[Errno 2] No such file or directory: 'drive/MyDrive/Structure-From-Motion-Accurate-Optical-Flow-vs-Active-Vision'
/content/drive/MyDrive/Structure-From-Motion-Accurate-Optical-Flow-vs-Active-Vision


/usr/local/lib/python3.8/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


#!/bin/bash
source /opt/ros/noetic/setup.bash
/opt/ros/noetic/bin/roscore

In [ ]:

if __name__ == '__main__':
    errors = []
    models = get_models()
    trials = get_trials()
    trials= np.array(trials).flatten()

    for algo_counter, model in enumerate(models):

        corner_counter = 0
        config_file =  model["config_f"]

        checkpoint_file = model["checkpoint_f"]
        mname = model["name"]
        gts = []  # Ground truth distances
        gts_ros=[]
        # Build the model from a config file and a checkpoint file
        #model = init_model(config_file, checkpoint_file, device='cuda:0')
        for counter, t in enumerate(trials):
            if t["trial_type"]!="gazelock":
              continue
  
            print(" Trial type: {}, Marker type {}".format(t["trial_type"],t["marker_type"] ))   
            rosbagf = "rosbags/{}.bag".format(t["trial"])
            # Convert the required Rosbag topics to dicts
            
            try:
              rospy.init_node("depth_estimation")
              buffer = tf2.Buffer(rospy.Duration(10000))
            except:
              print("Already registered")
            
     
            camera_infos = rosbag_to_dict(rosbagf, "/camera/color/camera_info")

            cam_poses = rosbag_to_dict(rosbagf, "/vrpn_client_node/pandactrl5_EE/pose")

            target_poses = rosbag_to_dict(rosbagf, "/vrpn_client_node/GCVS_largedist_target/pose")

            frames, frame_infos = rosbag_to_dataset(rosbagf, "/camera/color/image_raw/compressed",
                                                    "datasets/" + t["trial_type"] + "_" + t["marker_type"] + "_" + str(
                                                        t["gt"]) + "m")

            t_vels, r_vels, stime = rosbag_to_vel(rosbagf, ['/tf', '/tf_static'],
                                                  "/franka_state_controller/franka_states",buffer)

            frame_infos, frames = find_match(stime, frame_infos, frames)
            # Compute fps
            stamp1 = rospy.Time(secs=frame_infos[0]["header"]["stamp"]["secs"],
                                nsecs=frame_infos[0]["header"]["stamp"]["nsecs"])
            stamp2 = rospy.Time(secs=frame_infos[2]["header"]["stamp"]["secs"],
                                nsecs=frame_infos[2]["header"]["stamp"]["nsecs"])
            duration = (stamp2 - stamp1).to_sec()
            fps = 1 / duration
            camera_matrix = np.array(camera_infos[0]["K"]).reshape(3, 3)
            distortion_coefficients = np.array(camera_infos[0]["D"])
            focal_length = (camera_matrix[0][0] + camera_matrix[1][1]) / 2

            Z_mcenters = []  # Array for the depth of centers of the marks
            avg_depths = []  # Array for the average depth of the marks

            display_flow = False  # Whether or not to display the flow
            display_marker = False  # Whether or not to display the flow

            last_corners = np.array([])

            start_time = time.time()

            dists=[]
            # Ground-truth distances
            for j in range(len(target_poses)):
              t_p= np.array([[target_poses[j]["pose"]["position"]["x"]], [target_poses[j]["pose"]["position"]["y"]],[target_poses[j]["pose"]["position"]["z"]]])
              c_p= np.array([[cam_poses[j]["pose"]["position"]["x"]], [cam_poses[j]["pose"]["position"]["y"]],[cam_poses[j]["pose"]["position"]["z"]]])
              dists.append(abs(np.linalg.norm(t_p-c_p)))

            gt = np.mean(np.array(dists))
           
            if t["gt"] not in gts_ros:
              gts.append(gt)
              gts_ros.append(t["gt"] )
        
         
            for i in range(240): # was picked to be fair between trials since they contain different numbers of 
                # Calling only indices that allign with the poses (there are half as many poses as there are frames)
                img1 = frames[i * 2]
                img2 = frames[(i + 1) * 2]
                # Corner detection
                corners, succ = corner_detection(img1, img2, last_corners, i)
                # If no corner was detected and there is no alternative corner, skip iteration 
                if succ == False:
                    continue

                last_corners = corners

                # Optical flow inference: in case of gazelock it will be a zero array 
                flow = inference(model,mname, img1, img2, fps,True, t, i)
                
                mflow = marker_flow(corners ,flow ,t["trial_type"] ,img1.shape) # (pixel/s)

                avg_depth = depth_estimation(i, mflow, corners, t_vels, r_vels, focal_length,
                                                              img2.shape, t["trial_type"],camera_matrix)
                if avg_depth > 0:
                    avg_depths.append(avg_depth)
                   

            # Finished one trial
            avg_depths = iqr(1.0, avg_depths)
            trial_depth = sum(avg_depths) / len(avg_depths)
          
            trial_error = abs(trial_depth - gt)

            runtime = (time.time() - start_time)

            error_dict = {'gt': gt,
                          'marker_type': t["marker_type"],
                          'trial_type': t["trial_type"],
                          'mname': mname,
                          'error': round(trial_error, 3),
                          'stdev': np.std(avg_depths) /4, # for visualization only 
                          'runtime': runtime}
            errors.append(error_dict)
            print(error_dict)

    plot_all(gts_ros, errors)
  




 Trial type: gazelock, Marker type straight
Unable to register with master node [http://localhost:11311/]: master may not be running yet. Will keep trying.


/content/drive/MyDrive/Structure-From-Motion-Accurate-Optical-Flow-vs-Active-Vision/src/utils/utils_rosbag.py:79: UserWarning: translation should be of type Vector3
  buf.set_transform(msg_tf, "default_authority")
/content/drive/MyDrive/Structure-From-Motion-Accurate-Optical-Flow-vs-Active-Vision/src/utils/utils_rosbag.py:79: UserWarning: rotation should be of type Quaternion
  buf.set_transform(msg_tf, "default_authority")
/content/drive/MyDrive/Structure-From-Motion-Accurate-Optical-Flow-vs-Active-Vision/src/utils/utils_rosbag.py:77: UserWarning: translation should be of type Vector3
  buf.set_transform_static(msg_tf, "default_authority")
/content/drive/MyDrive/Structure-From-Motion-Accurate-Optical-Flow-vs-Active-Vision/src/utils/utils_rosbag.py:77: UserWarning: rotation should be of type Quaternion
  buf.set_transform_static(msg_tf, "default_authority")


Lookup would require extrapolation into the past.  Requested time 1635242311.772952080 but the earliest data is at time 1635242311.806994438, when looking up transform from frame [panda_link0] to frame [panda_EE]
Lookup would require extrapolation into the future.  Requested time 1635242320.073801279 but the latest data is at time 1635242320.062594891, when looking up transform from frame [panda_link0] to frame [panda_EE]
{'gt': 1.137396453216643, 'marker_type': 'straight', 'trial_type': 'gazelock', 'mname': 'PWC-NET ', 'error': 0.094, 'stdev': 0.0439279885902008, 'runtime': 7.779054641723633}
 Trial type: gazelock, Marker type straight
Lookup would require extrapolation into the past.  Requested time 1635243080.863307714 but the earliest data is at time 1635243080.897216320, when looking up transform from frame [panda_link0] to frame [panda_EE]
Lookup would require extrapolation into the future.  Requested time 1635243092.026248693 but the latest data is at time 1635243092.000094891, 

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd 

Mounted at /content/drive
